In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c89937438ba48af62f1ef3906fa78238e0bb0aaa4eb74fc34a85e455e7709593
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 760.8 kB/s eta 0:00:00
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [103]:
sc = SparkContext.getOrCreate()

In [104]:
rdd = sc.textFile('friendships.txt', 1)

In [105]:
def process_lines(line):
    try:
        user, friends = line.split("\t")
        user = int(user)
        friends = [int(friend) for friend in friends.split(",")]
        keys = []

        for friend in friends:
            keys.append((tuple(sorted([user, friend])), set(friends)))

        return keys
    except ValueError:
        return []

In [106]:
pairs_rdd = rdd.flatMap(process_lines)

In [107]:
# Find mutual friends
def map_mutual_friends(data):
    pair, friends = data
    return (pair, friends)

In [108]:
pairs_rdd = rdd.flatMap(process_lines)

In [109]:
def map_mutual_friends(data):
    pair, friends = data
    user, friend = pair
    mutual_friends = set(friends)
    return (pair, mutual_friends)

In [110]:
mapped_rdd = pairs_rdd.map(map_mutual_friends)

In [111]:
def reduce_mutual_friends(friends1, friends2):
    return friends1.intersection(friends2)

In [112]:
mutual_friends_rdd = mapped_rdd.reduceByKey(reduce_mutual_friends)

In [113]:
# Prepare recommendations
def recommend_mutual_friends(data):
    pair, mutual_friends = data
    user1, user2 = pair
    recommendations = []

    for mutual_friend in mutual_friends:
        recommendations.append((user1, mutual_friend))
        recommendations.append((user2, mutual_friend))

    return recommendations

In [114]:
recommendations_rdd = mutual_friends_rdd.flatMap(recommend_mutual_friends)

In [115]:
recommendation_counts = recommendations_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)

In [116]:
user_recommendations = recommendation_counts.map(lambda x: (x[0][0], (x[0][1], x[1])))

In [117]:
grouped_recommendations = user_recommendations.groupByKey().mapValues(list)

In [118]:
def suggest_top_mutual_friends(recommendations):
    user, friends_counts = recommendations
    top_mutual_friends = sorted(friends_counts, key=lambda x: -x[1])[:10]
    return (user, [friend for friend, count in top_mutual_friends])

In [119]:
suggestions = grouped_recommendations.map(suggest_top_mutual_friends)

In [120]:
output_rdd = suggestions.map(lambda x: f"{x[0]}\t{','.join(map(str, x[1]))}")

In [122]:
output_rdd.saveAsTextFile("mutualFriend_suggestions_output")